### DPS Test 방법 : 아래 각 코드를 터미널에서 입력해야함
1. `python bin/sparkapp.py korean_job --config_path=./configs/dedup_job_case_law.yaml` : yaml에서 config 바꿔줘야 함
2. `wc -l /home/seonghee_hong/legal_llm/dps/legal-project-data/DOMAIN_First_Dedup_CASE_LAW_0.8_16/part-00000` : 길이 확인

In [1]:
import os
import sys
import json
import random

# 필요한 경로 추가
base_path = '/home/seonghee_hong/legal_llm/dps'
sys.path.append(base_path)
sys.path.append(f'{base_path}/dps/spark')
sys.path.append(f'{base_path}/dps/spark/jobs')

from jobs.korean_job import *
from jobs.dedup_job import *
from prep.dedup_prep import *

In [2]:
sample_text = "이것은 테스트용 샘플 텍스트입니다. 1234와 같은 숫자 혹은 특수기호 *!@# 등이 포함될 수 있습니다."
print("Before Preprocess:", sample_text)

config_path = '/home/seonghee_hong/legal_llm/dps/configs/dedup_job_case_law.yaml'

with open(config_path) as f:
    conf = yaml.load(f, Loader=yaml.FullLoader)

# 위 필터 함수들을 한 번에 적용하는 함수 # ! 단순 테스트용임. 실제로는 사용 X
def total_filter(text):
    return bad_words_filter(text) and doc_len_filter(text, conf["min_doc_len"], conf["max_doc_len"]) and mean_word_len_filter(text, conf["min_mean_word_len"], conf["max_mean_word_len"]) and symbol_to_word_ratio_filter(text, conf["symbol_to_word_ratio"]) and bullet_ellipsis_filter(text, conf["bullet_point_ratio"], conf["ellipsis_ratio"]) and korean_word_ratio_filter(text, conf["korean_word_ratio"])

# 전처리 함수들을 한 번에 적용하는 함수 # ! 단순 테스트용임. 실제로는 사용 X
def total_preprocess(text): 
    if total_filter(text):
        return preprocess_text(make_compat(text))
    else:
        return None

total_preprocess_result = total_preprocess(sample_text)
print("Total preprocess result:", total_preprocess_result)

Before Preprocess: 이것은 테스트용 샘플 텍스트입니다. 1234와 같은 숫자 혹은 특수기호 *!@# 등이 포함될 수 있습니다.
Total preprocess result: 이것은 테스트용 샘플 텍스트입니다. 1234와 같은 숫자 혹은 특수기호 *!@# 등이 포함될 수 있습니다.


In [3]:
# 전처리 전 데이터 불러오기
file_path = "/home/seonghee_hong/legal_llm/dps/legal-project-data/CASE-LAW/sampled.jsonl"
with open(file_path, "r") as f:
    data = f.readlines()
    bf_data = [json.loads(line) for line in data]

bf_texts = [d["text"] for d in bf_data]

In [10]:
idx = random.randint(0, len(bf_texts))
print(f"Random index: {idx}")

# 전처리 전 데이터
print(f"=== Before preprocessing  === \n {bf_texts[idx]}")

# 전처리 후 데이터
print(f"=== After preprocessing === \n {total_preprocess(bf_texts[idx])}")


Random index: 19969
=== Before preprocessing  === 
 문서 종류 판결문 
 사건명 부당이득 
 사건제목 대법원 2009. 1. 15. 선고 2008도1246 판결 
 심판 유형 판례(대법원) 
 법원명 대법원 
 판결선고일 2009-01-15 
 대표사건번호 2008도1246 
 사건번호 2008도1246 
 판시사항 [1] 개발사업 등의 추진 전에 이를 알지 못하고 취득한 사업부지 일부를 사업자에게 매도하면서 시가보다 많은 대가를 약정·수령한 행위가 부당이득죄를 구성하는지 여부
[2] 아파트 신축사업이 추진되기 수년 전 사업부지 중 일부 토지를 취득하여 거주 또는 영업장소로 사용하던 피고인이 이를 사업자에게 매도하면서 시가 상승 등을 이유로 대금의 증액을 요구하여 종전보다 1.5 내지 3배가량 높은 대금을 받은 사안에서, 부당이득죄의 성립을 부정한 사례 
 질문 개발사업 등의 추진 전에 이를 알지 못하고 취득한 사업부지 일부를 사업자에게 매도하면서 시가보다 많은 대가를 약정·수령한 행위가 부당이득죄를 구성하는가? 
 답변 불명 
 요약대상원문 매도하라는 사업자 등의 제안을 받고 그 매도의 조건을 협상한 결과 큰 이득을 얻었다는 것만으로는 다른 특별한 사정이 없는 한 피해자의 궁박을 이용하여 현저하게 부당한 이익을 얻었다고 쉽사리 말할 수 없을 것이다. 그리고 그 협상의 과정에서 개발사업의 시행으로 인근 부동산의 시가가 전반적으로 상승한 것을 들어 대가의 증액을 요구했다고 해서 이를 형사적으로 비난받을 행태라고 할 수 없다. 피고인들은 이 사건 아파트신축사업이 추진되기 오래 전부터 위 각 부동산을 소유하여 오다가 대원피엠씨와의 위 매매계약에서 정한 매매대금으로 위 각 부동산을 매도하라는 피해자 회사의 제안을 거부하면서 피해자 회사와 사이에 새로 협상을 하여 종전의 매매대금보다 훨씬 많은 대금을 약정한 것으로서, 그것만으로는 피해자 회사의 궁박을 이용하여 현저하게 부당한 이익을 얻었다고 할 수 없다. 
 요약 문장 피고인들